In [ ]:
!pip install -q torch transformers transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
#load pdf files
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/RAG/iteration_2/database/")
ds = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [LangchainDocument(page_content=doc.page_content, metadata={"source": doc.metadata}) for doc in tqdm(ds)]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

  0%|          | 0/430 [00:00<?, ?it/s]

In [ ]:
from huggingface_hub import InferenceClient


repo_id = "meta-llama/Llama-2-7b-chat-hf"

llm_client = InferenceClient(
    model=repo_id,
    timeout=150,
)


def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 1000},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"]


call_llm(llm_client, "This is a test context")

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/meta-llama/Llama-2-7b-chat-hf (Request ID: CVYApV1xzFu3nHdnI1ElP)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate

In [ ]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [ ]:
import random

N_GENERATIONS = 200  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(llm_client, QA_generation_prompt.format(context=sampled_context.page_content))
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

Generating 200 QA couples...


  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
for output in tqdm(outputs):
    with open("output_gemma.json", "w") as f:
            json.dump(outputs, f)

  0%|          | 0/149 [00:00<?, ?it/s]

In [ ]:
 output_QA_couple

'\nYour task is to write a factoid question and an answer given a context.\nYour factoid question should be answerable with a specific, concise piece of factual information from the context.\nYour factoid question should be formulated in the same style as questions users could ask in a search engine.\nThis means that your factoid question MUST NOT mention something like "according to the passage" or "context".\n\nProvide your answer as follows:\n\nOutput:::\nFactoid question: (your factoid question)\nAnswer: (your answer to the factoid question)\n\nNow here is the context.\n\nContext: national recognized statistical rating organizations (NRSROs). \nDebt securities rated investment grade, that is those with ratings \nsimilar to BBB-/Baa3 or above, as defined by NRSROs, are \ngenerally considered by the rating agencies and market participants to be low credit risk. Conversely, debt securities \nrated below investment grade, labeled as “speculative grade” by \nthe rating agencies, are con

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(outputs, orient='columns')
df

,context,question,answer,source_doc
0,"RESIDENTIAL MORTGAGE LOANS Our residential mortgage loan \nportfolio is comprised of 1–4 family first and junior lien mortgage \nloans. Residential mortgage – first lien loans comprised 96% of \nthe total residential mortgage loan portfolio at September 30, \n2023 , compared with 95% at December 31, 2022 .\nThe residential mortgage loan portfolio includes loans with \nadjustable-rate features. We monitor the risk of default as a \nresult of interest rate increases on adjustable-rate mortgage \n(ARM) loans, which may be mitigated by product features that \nlimit the amount of the increase in the contractual interest rate. \nThe default risk of these loans is considered in our ACL for loans. \nARM loans were 7% of total loans at both September 30, 2023 , \nand December 31, 2022 , with an initial reset date in 2025 or later \nfor the majority of this portfolio at September 30, 2023 . We do \nnot offer option ARM products, nor do we offer variable-rate \nmortgage products with fixed payment amounts, commonly \nreferred to within the financial services industry as negative \namortizing mortgage loans.\nThe residential mortgage – junior lien portfolio consists of \nresidential mortgage lines of credit and loans that are \nsubordinate in rights to an existing lien on the same property. \nThese lines and loans may have draw periods, interest-only \npayments, balloon payments, adjustable rates and similar \nfeatures. The outstanding balance of residential mortgage lines \nof credit was $15.6 billion at September 30, 2023 , compared \nwith $18.3 billion at December 31, 2022 . The unfunded credit \ncommitments for these lines of credit totaled $30.1 billion at \nSeptember 30, 2023 , compared with $35.5 billion at \nDecember 31, 2022 . For additional information on our residential \nloan portfolio, see the “Risk Management – Credit Risk \nManagement – Residential Mortgage Loans” section in our 2022 \nForm 10-K . \nWe monitor changes in real estate values and underlying",(your factoid question)\n,(your answer to the factoid question)\n,"{'source': '/content/drive/MyDrive/RAG/iteration_2/database/q3_2023.pdf', 'page': 33}"
1,"Earnings Performance (continued) \nTable 6j: Corporate – Balance Sheet \nQuarter ended Mar 31, \n(in millions) 2023 2022 $ Change % Change \nSelected Balance Sheet Data (average) \nCash and due from banks, and interest-earning deposits with banks $ 117,419 178,747 (61,328) (34)% \nAvailable-for-sale debt securities (1) 128,770 156,756 (27,986) (18) \nHeld-to-maturity debt securities (1) 272,718 275,510 (2,792) (1) \nEquity securities 15,519 15,760 (241) (2) \nTotal loans 9,154 9,292 (138) (1) \nTotal assets 596,087 687,346 (91,259) (13) \nTotal deposits 60,807 27,039 33,768 125 \nSelected Balance Sheet Data (period-end) \nCash and due from banks, and interest-earning deposits with banks $ 136,093 175,201 (39,108) (22) \nAvailable-for-sale debt securities (1) 133,311 157,164 (23,853) (15) \nHeld-to-maturity debt securities (1) 274,202 277,965 (3,763) (1) \nEquity securities 15,200 16,137 (937) (6) \nTotal loans 9,247 9,101 146 2 \nTotal assets 620,241 682,912 (62,671) (9) \nTotal deposits 65,682 23,715 41,967 177 \n(1) In first quarter 2023, we reclassified HTM debt securities with a fair value of $23.2 billion to AFS debt securities in connection with the adoption of ASU 2022-01 – Derivatives and Hedging (Topic \n815): Fair Value Hedging – Portfolio Layer Method . For additional information, see Note 1 (Summary of Significant Accounting Policies) to Financial Statements in this Report. \nFirst quarter 2023 vs. first quarter 2022 \nTotal assets (average and period-end) decreased reflecting: \n• a decrease in cash and due from banks, and interest-earning \ndeposits with banks that are managed by corporate treasury",(your factoid question)\n,(your answer to the factoid question)\n,"{'source': '/content/drive/MyDrive/RAG/iteration_2/database/q1_2023.pdf', 'page': 22}"
2,"Testing” sect

In [ ]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [ ]:
question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independant this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independant from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [ ]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    with open("/content/drive/MyDrive/#RAG/output.json", "w") as f:
            json.dump(outputs, f)

    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]),
        ),
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        )
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
    except Exception as e:
        continue

Generating critique for each QA couple...


  0%|          | 0/185 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df.to_excel("test.xlsx")

In [ ]:
df.columns

Index(['context', 'question', 'answer', 'source_doc', 'groundedness_score',
       'groundedness_eval', 'relevance_score', 'relevance_eval'],
      dtype='object')

In [ ]:
df['standalone_score']=""
df['standalone_eval']=""

In [ ]:
for i in range(len(df)):
  evaluation=call_llm(
            llm_client,
            question_standalone_critique_prompt.format(question=df["question"][i]),
        )
  try:
    df['standalone_score'][i], df['standalone_eval'][i] = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
  except Exception as e:
        continue

<ipython-input-38-ecc47351f377>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['standalone_score'][i], df['standalone_eval'][i] = (
<ipython-input-38-ecc47351f377>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['standalone_score'][i], df['standalone_eval'][i] = (
<ipython-input-38-ecc47351f377>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['standalone_score'][i], df['standalone_eval'][i] = (
<ipython-input-38-ecc47351f3

In [ ]:
for i in range(48,len(df)):
  evaluation=call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=df["question"][i]),
        )
  try:
    df['relevance_score'][i], df['relevance_eval'][i] = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
  except Exception as e:
        continue

<ipython-input-26-107543af3918>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['relevance_score'][i], df['relevance_eval'][i] = (
<ipython-input-26-107543af3918>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['relevance_score'][i], df['relevance_eval'][i] = (
<ipython-input-26-107543af3918>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['relevance_score'][i], df['relevance_eval'][i] = (
<ipython-input-26-107543af3918>:7:

In [ ]:
for i in range(53,len(df)):
  evaluation=call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(context=df["context"][i], question=df["question"][i]),
        )
  try:
    df['groundedness_score'][i], df['groundedness_eval'][i] = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
  except Exception as e:
        continue



<ipython-input-23-c62bae9d50f2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['groundedness_score'][i], df['groundedness_eval'][i] = (
<ipython-input-23-c62bae9d50f2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['groundedness_score'][i], df['groundedness_eval'][i] = (
<ipython-input-23-c62bae9d50f2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['groundedness_score'][i], df['groundedness_eval'][i] = (
<ipython-input-2

In [ ]:
df

,context,question,answer,source_doc,groundedness_score,groundedness_eval,relevance_score,relevance_eval
0,"the periods presented. \nTable 13.2: Residential MSRs – Assumptions at Securitization Date \n2023 2022 \nQuarter ended June 30, \nPrepayment rate (1) 16.4 % 10.9 \nDiscount rate 9.4 8.0 \nCost to service ($ per loan) $ 176 122 \nSix months ended June 30, \nPrepayment rate (1) 17.5 % 11.0 \nDiscount rate 9.6 7.5 \nCost to service ($ per loan) $ 185 117 \n(1) Includes a blend of prepayment speeds and expected defaults. Prepayment speeds are \ninfluenced by mortgage interest rates as well as our estimation of drivers of borrower \nbehavior. \nSee Note 12 (Fair Values of Assets and Liabilities) and \nNote 6 (Mortgage Banking Activities) for additional information \non key assumptions for residential MSRs. RESECURITIZATION ACTIVITIES We enter into resecuritization \ntransactions as part of our trading activities to accommodate the \ninvestment and risk management activities of our customers. In \nresecuritization transactions, we transfer trading debt securities \nto VIEs in exchange for new beneficial interests that are sold to \nthird parties at or shortly after securitization settlement. This \nactivity is performed for customers seeking a specific return or \nrisk profile. Substantially all of our transactions involve the \nresecuritization of conforming mortgage-backed securities \nissued by the GSEs or guaranteed by GNMA. We do not \nconsolidate the resecuritization VIEs as we share in the decision-\nmaking power with third parties and do not hold significant \neconomic interests in the VIEs other than for market-making \nactivities. During the six months ended June 30, 2023 and 2022, \nwe transferred securities of $6.1 billion and $12.6 billion, \nrespectively, to resecuritization VIEs, and retained securities of \n$329 million and $525 million, respectively. These amounts are \nnot included in Table 13.1. Related total VIE assets were \n$111.1 billion and $112.0 billion at June 30, 2023, and \nDecember 31, 2022, respectively. As of June 30, 2023, and","What was the discount rate for the quarter ended June 30, 2023?\n","The discount rate for the quarter ended June 30, 2023 was 9.4.","{'source': '/content/docs/q2_2023.pdf', 'page': 116}",5.0,"The context provides the discount rate for the quarter ended June 30, 2023, which is 9.4. Therefore, the question is clearly and unambiguously answerable with the context.\n",1.0,"This question is not related to machine learning, natural language processing, or the Hugging Face ecosystem. It seems to be a financial question about a specific company's discount rate. Therefore, it is not useful for machine learning developers building NLP applications with the Hugging Face ecosystem.\n"
1,"Note 4: Equity Securities \nTable 4.1 provides a summary of our equity securities by business \npurpose and accounting method. \nTable 4.1: Equity Securities \n(in millions) Mar 31, \n2023 Dec 31, \n2022 \nHeld for trading at fair value: \nMarketable equity securities $ 12,764 17,180 \nNonmarketable equity securities (1) 10,513 9,730 \nTotal equity securities held for trading (2) 23,277 26,910 \nNot held for trading: \nFair value: \nMarketable equity securities 1,654 1,436 \nNonmarketable equity securities 72 37 \nTotal equity securities not held for trading at fair value 1,726 1,473 \nEquity method: \nPrivate equity 3,041 2,836 \nTax-advantaged renewable energy (3) 6,406 6,535 \nNew market tax credit and other 282 298 \nTotal equity method 9,729 9,669 \nOther methods: \nLow-income housing tax credit (LIHTC) investments (3) 12,151 12,186 \nPrivate equity (4) 9,028 9,276 \nFederal Reserve Bank stock and other at cost (5) 4,699 4,900 \nTotal equity securities not held for trading 37,333 37,504 \nTotal equity securities $ 60,610 64,414 \n(1) Represents securities economically hedged with equity derivatives. \n(2) Represents securities held as part of our customer accommodation trading activities. For a